### 0. 개요
이 코드 피일은 설정된 키워드를 기반으로 해당 키워드들이 사용된 게시글들을 DC인사이드 내에서 크롤링하여 제목과 본문을 csv형식으로 저장해주는 역할을 수행합니다.

### 1. 패키지 불러오기

In [4]:
import pandas as pd
from tqdm import tqdm
import requests 
from urllib import request
from bs4 import BeautifulSoup

### 2. 검색 키워드 설정

In [5]:
"""DC인사이드에서 검색하고자 하는 키워드를 아래 Array에 넣습니다. 추가한 단어들에 대해서 DC인사이드에서 최근 게시글 순으로 최대량을 검색하여 수집을 시작합니다."""
keywords = ['최저임금','최저시급', '최저 임금', '최저 시급']

### 3. 검색어를 이용한 페이지 크롤링

In [ ]:
"""앞서 입력한 키워드들마다, DC인사이드 내 검색 최대 페이지인 120번까지 검색을 진행합니다. 검색된 결과는 'pages' 배열에 (제목, URL)의 형태로 저장됩니다."""

pages = []
for keyword in keywords:
    print(keyword)
    for page_num in tqdm(range(1,121)):
        ## 페이지의 검색 메인 URL 입니다. 
        BASE_URL = "https://search.dcinside.com/post/p/%s/sort/latest/q/%s"%(str(page_num),keyword)
        headers = {"User-Agent": "owen (Android; Yusik_fuckyou)", "Cookie": "list_count=100"}
        
        # 검색창에 키워드를 입력하고, 각 검색페이지 번호를 돌며 문서들을 수집합니다.
        params = {}
        response = requests.get(BASE_URL, params=params ,headers=headers) 
        soup = BeautifulSoup(response.content, 'html.parser')

        #웹사이트의 경우 수시로 url이나 HTML 구조를 변경할 수 있으니, 유의하시기 바랍니다. 현재의 경우 html 구조 내에서 sch_result_list내에 검색된 정보를 저장하고 있습니다.
        html_list = soup.find(class_='sch_result_list').find_all('li')
        for i in html_list: 
            title_list = i.find('a', href=True).text # 제목 
            URL_list = i.find('a', href=True)['href'] # URL 주소 
            pages.append((title_list , URL_list))

### 4. 검색된 페이지 내의 내용 수집

In [ ]:
"""키워드를 통해 수집된 모든 페이지들을 돌며 해당 링크에 존재하는 게시글의 정보를 수집합니다."""

new_pages = []
for page in tqdm(pages[:40]):
    headers = {"User-Agent": "owen (Android; RandomID)", "Cookie": "list_count=100"}
    try: # request 에러로 검색이 실패하는 경우가 종종 있어 try문을 이용해 실패시에도 한번 더 요청하도록 구현합니다.
        response = requests.get(page[1], headers=headers) 
        soup = BeautifulSoup(response.content, 'html.parser')
        
        """11월 14일차 기준 본문의 경우 'script' 클래스였으나, HTML 구조의 변경으로 코드 재사용시 이 부분을 확인해야 합니다."""
        new_pages.append((page[0],soup.find('script').get_text()[17:-2]))
    except:
        try:
            response = requests.get(page[1], headers=headers) 
            soup = BeautifulSoup(response.content, 'html.parser')
            new_pages.append((page[0],soup.find('script').get_text()[17:-2]))
        except:
            continue


In [ ]:

"""앞서 수집한 페이지들을 순회하며 본문 내용을 [제목, 본문, 주소] 형태로 정리하여 재수집을 진행합니다. 수집된 데이터는 최종적으로 datas 배열에 저장됩니다."""

datas = []
for page in tqdm(new_pages):
    # 헤더 설정 
    headers = {"User-Agent": "owen (Android; RandomID)", "Cookie": "list_count=100"}
    # html 
    try: # request 에러로 검색이 실패하는 경우가 종종 있어 try문을 이용해 실패시에도 한번 더 요청하도록 구현합니다.
        response = requests.get(page[1], headers=headers) 
        soup = BeautifulSoup(response.content, 'html.parser')    
        
        """위와 마찬가지로 11월 14일차 기준 본문의 경우 'thum-txtin' 클래스였으나, HTML 구조의 변경으로 코드 재사용시 이 부분을 확인해야 합니다."""
        text_lines = soup.find(class_='thum-txtin').text
    except:
        try:
            response = requests.get(page[1], headers=headers[0]) 
            soup = BeautifulSoup(response.content, 'html.parser')    
            text_lines = soup.find('div',class_='thum-txtin').find_all('p')
        except:
            continue
    datas.append((page[0],text_lines,page[1]))

### 5. 수집된 데이터 저장

In [ ]:
"""위 코드들을 통해 수집된 DC인사이드 문서들을 전체글과 url 주소를 짝지어 CSV파일로 저장합니다."""
d = pd.DataFrame({'fulltext':[d[0]+d[1] for d in datas],
                 'url':[d[2] for d in datas]})

# 중복 되거나 누락된 글들은 필터링 됩니다.                 
data = data.dropna()
d = d.drop_duplicates(subset=['fulltext'], keep='first')

# csv 파일로 저장합니다.
d.to_csv('dcinside_search(%s).csv'%",".join(['"'+key+'"' for key in keywords]),index=False)